<a href="https://colab.research.google.com/github/DaniloTovar/Blockchain1/blob/main/Bitcoin_blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo de blockchain de Bitcoin
Ejercicio practico para replicar parcialmente las funcionalidades del blockchain de Bitcoin basado en el [white paper](https://bitcoin.org/bitcoin.pdf) del mismo.

Por favor ejecutar las celdas secuencialmente.

## Importación de librerias

In [1]:
import numpy as np
import hashlib as hlib
import itertools
import multipledispatch

## Funciones adicionales


In [2]:
# Funcion para realizar un hash con SHA-256 a un string
def hash(value):
  """
  Encrypts 'value' using a SHA-256 algorithm and returns the result
  """
  assert type(value) == str, "Wrong input type, 'value' should be a String"
  return hlib.sha256(value.encode()).hexdigest()

# Funcion para realizar un hash a cada elemento de a una lista de strings, utilizando la funcion 'hash'
def hashList(input):
  """
  Encrypts each element of 'input' using a SHA-256 algorithm
  """
  assert type(input) == list, "Wrong input type, 'input' should be a List"
  aux = []
  for i in input:
    try:
      aux.append(hash(i))
    except(AttributeError, TypeError):
      raise AssertionError("List should only contain Strings")
  return aux

# Realiza el procedimiento de Merkle Tree para generar un Merkle root a partir de una lista de strings 'Transaccion' utilizando la funcion 'hash' y 'hashList'
def getRoot(transactions):
  """
  Calculates the Merkle Root from a list of 'transaction' using a SHA-256 algorithm
  """
  assert type(transactions) == list, "Wrong input type, 'transactions' should be a list"
  assert len(transactions) != 0, "List of transactions cannot be empty"

  # Encontramos los hash iniciales de cada transaccion para luego empezar el proceso
  transactions = hashList(transactions)

  # Mientras no se halla encontrado el Merkle root, se va a seguir juntando pares
  while(len(transactions)!= 1):
    length = len(transactions)
    next = []

    # Si la lista es impar, se duplica el ultimo elemento de la lista
    if(np.mod(length,2)):
      last = transactions[-1]
      transactions.append(last)
      # print(last)

    # Se juntan pares y se les realizan hash
    for i in range(int(length/2)):
      string = transactions[i+i] + transactions[i+i+1]
      hashed = hash(string)
      next.append(hashed)
      # print(next)

    # Se define el siguiente nivel sobre el que trabajar
    transactions = next

  # Al finalizar se retorna el root
  return transactions[0]


# Funcion para generar numeros aleatorios positivos con limite superior en 'top'
def genNounce(top):
  """
  Generates a random number between 0 and 'top'
  """
  assert type(top) == int, "Wrong input type, 'top' should be a number"
  return int(np.random.random()*np.random.random()*top)

## Clases a utilizar para la construcción de la blockchain

In [3]:
# Clase Wallet: Representa la wallet, donde se almacenan las "llaves", una lista de las transacciones sin gastar (abiertas) y el balance total disponible
class Wallet:
  """
  A class that represents a bitcoin Wallet
  """

  # Inicializacion de la clase, donde se crea una instacia usando solo una "llave" privada.
  def __init__(self, private):
    """
    Class initializer that only needs the private key to use, it generates the public key, an empty open transactions list and starts with balance = 0
    """
    self.private = private
    self.public = hash(private)
    self.open_tx = []
    self.balance = 0

  # Funcion utilizada para imprimir en consola las caracteristicas de la cuenta
  def describe(self):
    """
    Function that prints the state of the wallet
    """
    return print("\t   public: ", self.public,"\n",
          "\t   available transactions: ", self.open_tx,"\n",
          "\t   balance: ", self.balance,"\n")

  # Funcion utilizada para obtener un string inmutable posteriormente utilizado en la generacion del hash del bloque cuando es añadido a la cadena
  # ('balance' y 'open_tx' pueden terminar generando un block hash diferente para el mismo bloque, por lo que no se tienen en cuenta)
  def toString(self):
    """
    Function that return a string used for the creation of the block hash, must be consistent after its been added to the chain
    """
    return str(self.public)

In [4]:
# Clase Output: Representa las salidas y mayoria de entradas de las transacciones, donde se almacenan la cantidad a enviar/utilizar,
# el destinatario (owner), en que transaccion se encuentra y si ha sido gastado por otra transaccion
class Output:
  """
  A class that represents a transaction output, also used for most transaction inputs
  """

  # Inicializacion de la clase, donde se crea una instacia usando la cantidad a enviar y el destinatario(owner).
  def __init__(self, amount, to):
    """
    Class initializer that needs the amount of bitcoins to send and the waller that is gonna recive them, defines 'id_tx' as -1 and 'spent' as False
    """
    assert type(to) == Wallet, "Wrong destination, should be a 'Wallet'"
    self.amount = amount
    self.to = to
    self.id_tx = -1
    self.spent = False

  # Funcion utilizada para imprimir en consola las caracteristicas del output
  def describe(self):
    """
    Function that prints the state of the output
    """
    print("\tamount: ", self.amount,"\n",
          "\tid_tx: ", self.id_tx,"\n",
          "\tspent: ", self.spent,"\n")
    print("\twallet: ")
    self.to.describe()
    print("\n")


  # Funcion utilizada para obtener un string inmutable posteriormente utilizado en la generacion del hash del bloque cuando es añadido a la cadena
  # (el estado de 'spent' pueden terminar generando un block hash diferente para el mismo bloque, por lo que no se tienen en cuenta)
  def toString(self):
    """
    Function that return a string used for the creation of the block hash, must be consistent after its been added to the chain
    """
    return str(self.amount) + (self.to.toString()) + str(self.id_tx)

In [5]:
# Clase Transaccion: Representa una transaccion y su realizacion, donde se almacenan una id para la transaccion, una lista de inputs,
# una lista de outputs y el hash del bloque en el que se almacenó la transaccion una vez fué agregado a la cadena
class Transaction:
  """
  A class that represents a transaction between wallets
  """

  # Id autoincremental comenzando en 1
  id_tx = itertools.count(1)


  # Constructor de una transaccion normal
  @multipledispatch.dispatch(inputs = list, outputs = list)
  def __init__(self, inputs, outputs):
    """
    Class initializer that needs a list with the inputs (previous transaction output/s) and a list with the new outputs of the transaction.
    Generates a self-increasing id and defines 'block_hash' as -1
    """
    # Se genera la id para realizar verificaciones
    new_id = next(Transaction.id_tx)

    # Verificaciones sobre los parametros de entrada
    assert all(((type(ins) == Output) & (ins.id_tx != new_id) & (ins.spent == False)) for ins in inputs), "List of inputs should only contain previous unspent 'Output' objects"
    assert all(((type(out) == Output) & (out.id_tx == -1)) for out in outputs), "List of outputs should only contain new 'Output' objects"

    # Primero, validamos que si se pueda realizar la transaccion
    try:
      check = []
      check2 = []
      for out in outputs:
        check.append(out.amount)
      for ins in inputs:
        check2.append(ins.amount)
      check = sum(check)   # Suma todos los amount de outputs
      check2 = sum(check2)     # Suma todos los amount de inputs
      if(check > check2):
        raise ArithmeticError("Amount to output is higher than input amount")
    except:
      raise ArithmeticError("Sum of operations is invalid")

    # Si la transaccion es valida, procedemos a realizar la transaccion
    self.id = new_id
    self.inputs = inputs
    self.outputs = outputs
    self.block_hash = -1

    # Actualizamos las wallets
      # Actualizamos los balances
    for input in self.inputs:
      input.to.balance -= input.amount
    for output in self.outputs:
      output.to.balance += output.amount

      # Actualizamos la lista de trasacciones abiertas
    for input in self.inputs:
      input.to.open_tx.remove(input.id_tx)
    for output in self.outputs:
      output.to.open_tx.append(self.id)


    for ins in inputs:
      ins.spent = True        # Actualizamos los inputs como 'spent' cuando se genera esta transaccion

    for out in outputs:
      out.id_tx = self.id     # Actualizamos los outputs para relacionarlos a esta transaccion



  # Constructor de un CoinBase
  @multipledispatch.dispatch(Wallet)
  def __init__(self, wallet):
    """
    Class initializer that only needs a 'Wallet' destination, used to generate a CoinBase of 50 for that wallet
    """
    self.id = next(Transaction.id_tx)
    self.inputs = [0]
    output0 = Output(50, wallet)
    self.outputs = [output0]
    self.block_hash = -1

    self.outputs[0].to.balance += self.outputs[0].amount  # Actualizamos el balance de la wallet
    self.outputs[0].to.open_tx.append(self.id)     # Actualizamos la lista de transacciones abiertas relacionadas a la wallet
    self.outputs[0].id_tx = self.id     # Actualizamos el output para relacionarlo a esta transaccion

  # Funcion utilizada para imprimir en consola las caracteristicas de la transaccion
  def describe(self):
    """
    Function that prints the state of the transaction
    """
    print("id: ", self.id,"\n",
          "block hash: ", self.block_hash,"\n",
          "inputs:")
    for ins in self.inputs:
      try:
        ins.describe()
      except:
        print("\tInput not found")
    print("outputs:")
    for out in self.outputs:
      out.describe()

  # Funcion utilizada para obtener un string inmutable posteriormente utilizado en la generacion del hash del bloque cuando es añadido a la cadena
  # (el 'block_hash' se almacena una vez el bloque es añadido a la cadena, por lo que no se tienen en cuenta)
  def toString(self):
    """
    Function that return a string used for the creation of the block hash, must be consistent after its been added to the chain
    """
    aux1 = ""
    for ins in self.inputs:
      try:
        aux1  = aux1 + (ins.toString())
      except(AttributeError):
        aux1 = aux1 + str(ins)
    aux2 = ""
    for out in self.outputs:
      aux2 = aux2 + (out.toString())
    return str(self.id) + aux1 + aux2

In [6]:
# Clase Bloque: Representa un bloque por añadir o añadido a la cadena, donde se almacenan las transacciones, el merkle root,
# el hash del bloque previo y el nounce del bloque
class Block:
  """
  A class that represents a block from bitcoin
  """

  # Constructor del bloque, requiere la lista de objetos 'Transation' a almacenar y el hash del bloque previo
  def __init__(self, transactions, prev_hash):
    """
    Class initializer that needs a list of 'Transaction' and the previous block hash, as the Merkle root is automatically generated
    and starts with a random nounce with 'top' 10^10
    """

    # Verificaciones sobre los parametros de entrada
    assert all((type(tx) == Transaction) for tx in transactions), "List of transactions should only contain 'Trasaction' objects"


    # Obtenemos los strings inmutables de las transacciones para generar posteriormente generar el merkle root
    str_transactions = []
    for tx in transactions:
      str_transactions.append(tx.toString())

    self.transactions = transactions
    self.merkle_root = getRoot(str_transactions)
    self.prev_hash = prev_hash
    self.nounce = genNounce(10**10)


  # Funcion para generar el hash del bloque
  def genHash(self):
    """
    Function that returns the hash of the block
    """
    str_self = str(self.merkle_root) + str(self.prev_hash) + str(self.nounce)
    return hash(str_self)

  # Funcion utilizada para validar la totalidad de las transacciones realizadas en el bloque
  # (opcional : debido a que se hace la verificacion individual cuando se crean las transacciones)
  def Validate(self):
    for tx in self.transactions:
      aux = []
      aux2 = []
      for out in tx.outputs[1:]:
        aux.append(out.amount)    # Suma todos los valores de outputs excepto el CoinBase
      for ins in tx.inputs[1:]:
        aux2.append(ins.amount)   # Suma todos los valores de inputs excepto el CoinBase
      out_total_amount = sum(aux)
      in_total_amount = sum(aux2)
      return in_total_amount <= out_total_amount


  # Funcion utilizada para imprimir en consola las caracteristicas del bloque
  def describe(self):
    """
    Function that prints the state of the block
    """
    print("Header: \n",
          " prev_hash: ", self.prev_hash,"\n",
          " merkle_root: ", self.merkle_root,"\n",
          " nounce: ", self.nounce,"\n",
          "\nBody: \n",
          "Transactions: ")
    for tx in self.transactions:
      print("\t")
      tx.describe()

In [7]:
# Clase Cadena: Representa una cadena de bloques, inicialmente con una lista vacia
class Chain:
  """
  A class that represents a chain of 'Block's
  """

  # Constructor de la cadena
  def __init__(self):
    """
    Class initializer, creates an atribute with an empty list of blocks
    """
    self.chain = []

  # Funcion para añadir un bloque a la cadena, donde se realiza la prueba de trabajo y se actualizan las transacciones con el hash final del bloque
  def addBlock(self, block, difficulty_level):
    """
    Function used to add a block to the chain, where the "proof of work" is done and the transactions state is updated
    """

    # Verificaciones sobre los parametros de entrada
    assert type(difficulty_level) == int, "Wrong type, 'difficulty_level' should be a integer"
    assert difficulty_level >= 0, "'difficulty_level' should not be negative"
    assert type(block) == Block, "Wrong type, 'block' should be a Block"


    # Validacion sobre la totalidad del bloque (opcional)
    assert block.Validate() == True, "Invalid block, sum of transaction operations doesnt match"

    # Proof of work
    goal = str("0"*difficulty_level)
    hashed_nounce = hash(str(block.nounce))
    while(not(hashed_nounce.startswith(goal))):
      block.nounce = genNounce(10**10)
      hashed_nounce = hash(str(block.nounce))

    print(goal,"? -->", hashed_nounce, "with", block.nounce)
    self.chain.append(block)      # Agregamos el bloque en la cadena
    print("Block added")

    # Actualizamos las transacciones con el hash del bloque en el que quedaron
    for tx in block.transactions:
      tx.block_hash = block.genHash()


## Demostración de la blockchain

En caso de modificar las funciones y clases en la parte superior volver a iniciar desde este punto para evitar error inesperados.

Creación de Wallets vacias

In [8]:
Wallet1 = Wallet("Pepito123456")
Wallet2 = Wallet("Edison147852")
Wallet3 = Wallet("Fernando987")
Wallet4 = Wallet("Ricardo456456")

Wallet1.describe()
Wallet2.describe()
Wallet3.describe()
Wallet4.describe()

	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [] 
 	   balance:  0 

	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [] 
 	   balance:  0 

	   public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 	   available transactions:  [] 
 	   balance:  0 

	   public:  5bf8162f54797354bba873f473f0065ce5108e21cee3870d069aba4ad37eaa21 
 	   available transactions:  [] 
 	   balance:  0 



Creación de un conjunto de transacciones

In [9]:
ttx0 = Transaction(Wallet1)
ttx0.describe()

id:  1 
 block hash:  -1 
 inputs:
	Input not found
outputs:
	amount:  50 
 	id_tx:  1 
 	spent:  False 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [1] 
 	   balance:  50 





In [10]:
# Observamos como se ve reflejada la transaccion anterior en la wallet 1
Wallet1.describe()

	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [1] 
 	   balance:  50 



In [11]:
# Si se intenta realizar una transaccion invalida, se genera un error y no se aplica la transaccion
# Input = 50, Output = 20,40 = 60 ; 50 < 60 --> La wallet 1 no tiene suficientes bitcoin

ttx1 = Transaction(inputs=[ttx0.outputs[0]],outputs=[Output(20,Wallet2), Output(40,Wallet1)])
ttx1.describe()

ArithmeticError: Sum of operations is invalid

In [12]:
# Podemos observar que no se modificó la información de la
Wallet1.describe()
Wallet2.describe()

	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [1] 
 	   balance:  50 

	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [] 
 	   balance:  0 



In [13]:
# Si intento realizar una transaccion valida, se aplican los cambios necesarios
ttx1 = Transaction(inputs=[ttx0.outputs[0]],outputs=[Output(20,Wallet2), Output(30,Wallet1)])
ttx1.describe()

id:  3 
 block hash:  -1 
 inputs:
	amount:  50 
 	id_tx:  1 
 	spent:  True 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 



outputs:
	amount:  20 
 	id_tx:  3 
 	spent:  False 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [3] 
 	   balance:  20 



	amount:  30 
 	id_tx:  3 
 	spent:  False 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 





In [14]:
# Vemos como se actualizan las wallets involucradas
Wallet1.describe()
Wallet2.describe()

	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 

	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [3] 
 	   balance:  20 



In [15]:
# Y que la transaccion utilizada como input fue actualizada para reflejar que ya fue gastada ('spent')
ttx0.describe()

id:  1 
 block hash:  -1 
 inputs:
	Input not found
outputs:
	amount:  50 
 	id_tx:  1 
 	spent:  True 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 





Creación de un bloque 'inicial'

In [16]:
Block0 = Block([ttx0, ttx1], hash(str("prev_hash")))
Block0.describe()

Header: 
  prev_hash:  009166c917b09f86aba0d1f2dc1c6c6b874453b0224a7d505a16069bc6593181 
  merkle_root:  f5cc8ec3628c0e550e54119f8bb804d5575e6e4812b61bd10e6c97879b8476cf 
  nounce:  1910723005 
 
Body: 
 Transactions: 
	
id:  1 
 block hash:  -1 
 inputs:
	Input not found
outputs:
	amount:  50 
 	id_tx:  1 
 	spent:  True 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 



	
id:  3 
 block hash:  -1 
 inputs:
	amount:  50 
 	id_tx:  1 
 	spent:  True 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 



outputs:
	amount:  20 
 	id_tx:  3 
 	spent:  False 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [3] 
 	   balance:  20 



	amount:  30 
 	id_tx:  3 
 	spent:  False 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db0

In [17]:
# Validación sobre todas las transacciones ocurridas en el bloque, debe retornar True si el bloque es válido
Block0.Validate()

True

Cración de una cadena vacía

In [18]:
long_chain = Chain()
long_chain.chain

[]

Añadimos el bloque a la cadena creada

In [19]:
# Block, difficulty
long_chain.addBlock(Block0, 5)

00000 ? --> 000003ab00e8fd908a4febe6043adb221e73b6bbcb05d3a893cd43e1c70701ca with 478253051
Block added


In [20]:
long_chain.chain[0].describe()

Header: 
  prev_hash:  009166c917b09f86aba0d1f2dc1c6c6b874453b0224a7d505a16069bc6593181 
  merkle_root:  f5cc8ec3628c0e550e54119f8bb804d5575e6e4812b61bd10e6c97879b8476cf 
  nounce:  478253051 
 
Body: 
 Transactions: 
	
id:  1 
 block hash:  aaf0cac0f79346f79aa9b4b8126c254cee80871b812cdc10c9622098aa60b725 
 inputs:
	Input not found
outputs:
	amount:  50 
 	id_tx:  1 
 	spent:  True 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 



	
id:  3 
 block hash:  aaf0cac0f79346f79aa9b4b8126c254cee80871b812cdc10c9622098aa60b725 
 inputs:
	amount:  50 
 	id_tx:  1 
 	spent:  True 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3] 
 	   balance:  30 



outputs:
	amount:  20 
 	id_tx:  3 
 	spent:  False 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [3] 


## Más interacción en la blockchain

In [21]:
ttx2 = Transaction(Wallet3)
ttx2.describe()

id:  4 
 block hash:  -1 
 inputs:
	Input not found
outputs:
	amount:  50 
 	id_tx:  4 
 	spent:  False 

	wallet: 
	   public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 	   available transactions:  [4] 
 	   balance:  50 





In [22]:
ttx3 = Transaction(inputs=[ttx2.outputs[0]],outputs=[Output(20,Wallet2), Output(30,Wallet1)])
ttx3.describe()

id:  5 
 block hash:  -1 
 inputs:
	amount:  50 
 	id_tx:  4 
 	spent:  True 

	wallet: 
	   public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 	   available transactions:  [] 
 	   balance:  0 



outputs:
	amount:  20 
 	id_tx:  5 
 	spent:  False 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [3, 5] 
 	   balance:  40 



	amount:  30 
 	id_tx:  5 
 	spent:  False 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3, 5] 
 	   balance:  60 





In [23]:
Wallet2.describe()

	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [3, 5] 
 	   balance:  40 



In [24]:
ttx4 = Transaction(inputs=[ttx1.outputs[0], ttx3.outputs[0]],outputs=[Output(20,Wallet4), Output(20,Wallet1)])

In [25]:
ttx4.describe()

id:  6 
 block hash:  -1 
 inputs:
	amount:  20 
 	id_tx:  3 
 	spent:  True 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [] 
 	   balance:  0 



	amount:  20 
 	id_tx:  5 
 	spent:  True 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [] 
 	   balance:  0 



outputs:
	amount:  20 
 	id_tx:  6 
 	spent:  False 

	wallet: 
	   public:  5bf8162f54797354bba873f473f0065ce5108e21cee3870d069aba4ad37eaa21 
 	   available transactions:  [6] 
 	   balance:  20 



	amount:  20 
 	id_tx:  6 
 	spent:  False 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 	   available transactions:  [3, 5, 6] 
 	   balance:  80 





In [26]:
Block1 = Block([ttx2,ttx3,ttx4], Block0.genHash())
Block1.describe()

Header: 
  prev_hash:  aaf0cac0f79346f79aa9b4b8126c254cee80871b812cdc10c9622098aa60b725 
  merkle_root:  7a5d2983f416311b222c3d10089f54022856f395951c3549fb9013a1177e2867 
  nounce:  3720664311 
 
Body: 
 Transactions: 
	
id:  4 
 block hash:  -1 
 inputs:
	Input not found
outputs:
	amount:  50 
 	id_tx:  4 
 	spent:  True 

	wallet: 
	   public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 	   available transactions:  [] 
 	   balance:  0 



	
id:  5 
 block hash:  -1 
 inputs:
	amount:  50 
 	id_tx:  4 
 	spent:  True 

	wallet: 
	   public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 	   available transactions:  [] 
 	   balance:  0 



outputs:
	amount:  20 
 	id_tx:  5 
 	spent:  True 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [] 
 	   balance:  0 



	amount:  30 
 	id_tx:  5 
 	spent:  False 

	wallet: 
	   public:  1217d76e89796d523a61815fb7d198db01d6c793

In [27]:
long_chain.addBlock(Block1, 5)

00000 ? --> 00000b5e900593fd3d38460311434c6b5a251744ecc9e4d726c5b472d197b2ce with 1985006597
Block added


In [28]:
long_chain.chain

[<__main__.Block at 0x78e14021cb80>, <__main__.Block at 0x78e14021ea70>]

In [29]:
long_chain.chain[1].describe()

Header: 
  prev_hash:  aaf0cac0f79346f79aa9b4b8126c254cee80871b812cdc10c9622098aa60b725 
  merkle_root:  7a5d2983f416311b222c3d10089f54022856f395951c3549fb9013a1177e2867 
  nounce:  1985006597 
 
Body: 
 Transactions: 
	
id:  4 
 block hash:  ef7df8ed2f80c0afc2aa7c69411501764dcbc439c6ea5f1bce63065df9744bd5 
 inputs:
	Input not found
outputs:
	amount:  50 
 	id_tx:  4 
 	spent:  True 

	wallet: 
	   public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 	   available transactions:  [] 
 	   balance:  0 



	
id:  5 
 block hash:  ef7df8ed2f80c0afc2aa7c69411501764dcbc439c6ea5f1bce63065df9744bd5 
 inputs:
	amount:  50 
 	id_tx:  4 
 	spent:  True 

	wallet: 
	   public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 	   available transactions:  [] 
 	   balance:  0 



outputs:
	amount:  20 
 	id_tx:  5 
 	spent:  True 

	wallet: 
	   public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 	   available transactions:  [] 
 	   